In [1]:
#Importing packages
import requests
import bs4 as bs
import pandas as pd
import re
import datetime
from datetime import datetime
# Getting the URL
url = 'https://en.wikipedia.org/wiki/List_of_post%E2%80%932016_election_Donald_Trump_rallies'
response = requests.get(url, headers={'name': 'Laurits Harbom Marcussen', 'email': 'zhg932@alumni.ku.dk'})
soup = bs.BeautifulSoup(response.content, 'lxml') # Get the HTML of the website
tables = soup.find_all('table')

In [2]:
#Converts html's to dataframes, convert lists to dataframe
df=pd.read_html(str(tables))
df1=pd.DataFrame(df[8])
df2=pd.DataFrame(df[9])

#Isolate the "Date of rally2 columns, and combine the 2 tables
dates = pd.concat([df1["Date of rally"], df2["Date of rally"]])
dates = dates.reset_index(drop=True)

#Cleans wiki-data
def clean():
    for i in range(len(dates)):
        dates[i] = re.sub(r'\[\d+\]', '', dates[i])
        date_list = dates[i].split(',') # # Removing the weekday and the comma from the strings by splitting the strings and merging them again
        dates[i] = ''.join(date_list[1:])
        dates[i] = pd.to_datetime(dates[i]).strftime('%Y-%m-%d') # Converting to datetime format
    return dates
clean()

0     2020-06-20
1     2020-06-23
2     2020-08-17
3     2020-08-17
4     2020-08-18
         ...    
65    2020-11-02
66    2020-11-02
67    2020-12-05
68    2021-01-04
69    2021-01-06
Name: Date of rally, Length: 70, dtype: object

In [3]:
#Craeting a list of all the dates we wanna analyze
all_dates = pd.date_range(start='6/01/2020', end='2/01/2021')
date_list = all_dates.strftime('%Y-%m-%d')
s = pd.Series(date_list)

In [4]:
#Comparing the list of all dates and the list of rally dates, and making a binary list of coinciding dates
binary_list = []
for i in s:
    if i in dates.values: 
        binary_list.append(1)
    else: 
        binary_list.append(0)
binaries = pd.Series(binary_list)